In [98]:
import numpy as np

# Function created to initialize the Item vector - implement one that gets the data from a .txt file
# https://awari.com.br/vetor-python-aprenda-a-utilizar-a-linguagem-de-programacao-para-manipular-vetores/
def askForItens():
    qntd_itens = int(input("Type the number of items:"))
    #Create maze with dimension 2xItems    
    matriz = np.zeros((2, qntd_itens))
    
    ## Fill Valor
    for i in range(qntd_itens):
        matriz[0][i] = int(input(f"Type the Value for each item {i+1}:"))
    ## Fill Peso
    for i in range(qntd_itens):
        matriz[1, i] = float(input(f"Type the Weight for each item {i+1}: "))
    return matriz

In [99]:
import numpy as np

def mochilaInicial(n):
    # Gerando valores aleatórios para pesos e valores dos itens
    pesos = np.random.randint(0, 16, size=n)
    valores = np.random.randint(0, 16, size=n)

    # Construindo a matriz com pesos e valores
    matriz = np.array([pesos, valores])

    return matriz

def capacidadeMochila(matriz):
    somaTotal = np.sum(matriz[0])
    percentage = 0.5
    capacidade = int(somaTotal * percentage)
    return capacidade


In [100]:
# Fornecer uma opção randomica de itens que estariam inseridos nessa primeira lista de solução
def initRandom(size):
    # 100% random! 
    randomVector = np.random.choice([True, False], size=size)
    return randomVector

In [101]:
import random
import math

# Calcular o valor total a mochila com os itens dentro dela
def calculate_value(solution, original, capacity):
    total_weight = 0
    total_value = 0

    # Loop para percorrer o vetor inteiro
    for i in range(len(solution)):
        # Calcular o peso e valor total para cada item que estiver presente 
        if(solution[i] == 1):
            # Coluna o item que foi selecionado
            total_weight += original[0][i]
            total_value += original[1][i]
            # Desconsidera se tiver ultrapassado a capacidade
            if(total_weight > capacity):
                # Retorna um valor negativo para indicar que estourou
                return 0
    return total_value

In [102]:
def RandomNeighbor(solution):
    # [:] é utilizado para criar uma cópia sem referenciar o mesmo espaço de memória
    # já que, se for apontando os 2 para o mesmo lugar, se eu alterar altera ambos - PARA VETORES
    tempSolution = solution[:]
    # Ver quais itens estarão na mochila - pelo índice 
    index = random.randint(0, len(solution) - 1)
    # 1 - 0 = 1  e  1 - 1 = 0
    tempSolution[index] = 1 - tempSolution[index]  # inverte o valor de 0 para 1 ou de 1 para 0
    return tempSolution

In [103]:
import copy
def solveKnapsack():
    # Perguntar a capacidade total da Mochila
    # capacity = int(input("Bag Capacity: "))
    #capacity = 20


    # Comecar perguntando sobre os items
    #originalValues = askForItens()
    #originalValues = np.array([
    #[2, 3, 4, 5, 9, 7, 1, 6, 4, 8, 3, 5],  # Pesos
    #[3, 4, 5, 8, 10, 7, 2, 9, 6, 12, 5, 7]  # Valores
    #[12, 7, 11, 8, 9, 15, 6, 4, 13, 8, 12, 9, 7, 5, 11, 6, 10, 4, 9, 7],
    #[24, 13, 23, 15, 16, 30, 10, 8, 25, 20, 24, 17, 14, 12, 23, 9, 21, 7, 19, 13]
    #])
    
    #Quantidade de Itens 
    n = 100
    originalValues = mochilaInicial(n)
    print(originalValues)
    capacity = capacidadeMochila(originalValues)
    print(capacidadeMochila(originalValues))

    # Definir estados Iniciais - Aleatório
    currentSolution = initRandom(originalValues.shape[1])
    currentValue = calculate_value(currentSolution, originalValues, capacity)
    #print(f"Vetor Inicial: {initialSolution}")
    #print(f"Valor Atual: {currentValue}")

    # A melhor solução é a que se tem no momento
    bestSolution = currentSolution [:]
    bestValue = currentValue

    # Temperatura inicial
    # Temperatura recebe um valor "aleatório" que vai dimiuindo conforme aumentar o número e iterações
    temperature = 10000
    #decreaseFactor = 1
    decreaseFactor = 0.99
    stopTemp = 5

    # Stuck count para "sair" de loops ruins
    stuckCount = 0
    maxStuckCount = 1000

    igualZero = 0 
    totalIteracoes = 0
    countPior = 0

    # Loop infinito - O critério de parada é se não vai aceitar mais variações ("Temperatura"<=0)
    # Loop infinito - O critério de parada é se não vai aceitar mais variações ("Temperatura"<=0)
    while temperature > stopTemp:
        # Ver Neighbor
        Neighbor = RandomNeighbor(currentSolution)
        valorNeighbor = calculate_value(Neighbor, originalValues, capacity)
        
        if valorNeighbor != 0:    
            # Fit = Próximo - Atual > para ver se é melhor que a atual
            fitness = valorNeighbor - currentValue
            totalIteracoes= totalIteracoes + 1
            # Se Valor Neighbor for MAIOR que o valor ATUAL
            if fitness > 0:
                # Se for MAIOR: Atual = Próximo
                currentSolution = copy.deepcopy(Neighbor)
                currentValue = valorNeighbor
            # Se NÃO for MAIOR
            else:
                # Se existe a probabilidade de aceitar
                if random.uniform(0, 1) < math.exp(fitness / temperature):
                    # Então Atual = Próximo
                    countPior = countPior + 1
                    currentSolution = copy.deepcopy(Neighbor)
                    currentValue = valorNeighbor
            
            # Guarda o maior
            if currentValue > bestValue:
                bestValue = currentValue
                bestSolution = copy.deepcopy(currentSolution)

            temperature *= decreaseFactor
            #temperature = temperature - decreaseFactor
        else:
            igualZero = igualZero + 1
    print(f"Total de iteracoes validas: {totalIteracoes}")
    print(f"Total de iteracoes invalidas: {igualZero}")
    print(f"Aceitou o pior: {countPior}")
    return bestSolution, bestValue


In [104]:
bestSolution, bestValue = solveKnapsack()
print(f'Melhor solução: {bestSolution} com valor {bestValue}')

[[15 12  6  6  5  0 11 12 14  4 13 13 11  6  2 12  3 12  2  6  1  1  7  4
   9 13  0 13  9 11  7  1 10 15 11 15 10 11  5  2  3  0  5  3  6  1  8  3
   0  3  1  7  9 10  2  1 14 13 15 15  0  0  9 11  4 12 10  1 11  4  7 11
   1 12 11  6  2 15  2  9  1  0 14  4 12 13 15  1  5  1  9  5 10  7  0  3
   7 10 11  9]
 [10 15  2  4  8  6 14 15 14  4 13  5  7  4  8 10 13  8 11  9  8 14  9 13
   5 15  7 15  1  0  7  8 15  9  4 11  4 14 13  7 14  6  5  6 13 12 11  3
   8  8  3  5  2  6  5  5  0  1  2  4 10 12 11  2 13 15 13 14  9 15 11 12
   9  4  0  6  9  2  7 11  5  0 10 12  6  6  9 10  3  8  1  7  8  0  7 12
   2  5 12  7]]
357
Total de iteracoes validas: 757
Total de iteracoes invalidas: 427
Aceitou o pior: 336
Melhor solução: [ True False  True  True False False  True False False  True  True False
 False  True  True  True False  True False False  True  True  True False
 False  True  True False  True False  True False  True  True  True False
 False False False  True  True False False  True  Tr

In [105]:
O problema que você estava enfrentando com o código estava relacionado ao modo como o Python lida com referências de listas.

Explicando:
No Python, quando você faz algo como bestSolution = currentSolution[:], você está criando uma cópia superficial (shallow copy) da lista currentSolution. Ou seja, você cria uma nova lista, mas essa nova lista compartilha os mesmos objetos que a lista original. Isso funciona bem quando os elementos da lista são valores simples, como números. Porém, em alguns casos, se os objetos na lista forem mais complexos (por exemplo, listas dentro de listas ou objetos mutáveis), esse tipo de cópia pode acabar criando referências indesejadas.

No seu caso, como estamos lidando com listas que representam soluções, fazer uma cópia superficial com [:] pode ainda manter uma conexão entre bestSolution e currentSolution. Se você modifica currentSolution depois, as alterações podem, acidentalmente, refletir também em bestSolution, já que as duas soluções podem ainda estar referenciando os mesmos objetos. Isso explica o comportamento inesperado de bestSolution mudando no final do algoritmo.

Cópia superficial vs. Cópia profunda:
Cópia superficial ([:]): Cria uma nova lista, mas os elementos da nova lista são apenas referências aos objetos originais. Isso pode ser problemático se esses objetos forem mutáveis.

Exemplo:

python
Copiar código
lista1 = [1, 2, 3]
lista2 = lista1[:]
lista1[0] = 99
# lista2[0] ainda será 1, pois a lista contém apenas valores simples (números).
Cópia profunda (copy.deepcopy): Cria uma nova lista e também copia todos os objetos internos, criando cópias completas e independentes. Isso garante que mesmo que você altere a nova lista ou os objetos dentro dela, a original permanecerá inalterada.

Exemplo:

python
Copiar código
import copy
lista1 = [[1, 2], [3, 4]]
lista2 = copy.deepcopy(lista1)
lista1[0][0] = 99
# lista2[0][0] ainda será 1, porque agora cada lista interna também foi copiada.
O Problema no Algoritmo de Têmpera Simulada:
Na têmpera simulada, o algoritmo tenta explorar diferentes soluções (currentSolution) e manter a melhor solução encontrada até o momento (bestSolution). No entanto, ao usar [:] para copiar currentSolution para bestSolution, você estava criando uma cópia superficial. Isso funcionava bem até o ponto em que currentSolution era alterada mais tarde no algoritmo, levando a alterações inesperadas também em bestSolution, já que ambos ainda estavam "conectados" por referências internas.

Isso fazia parecer que bestSolution estava sendo alterada no final do algoritmo, quando, na verdade, ela estava sendo modificada indiretamente quando currentSolution era atualizada.

Solução com deepcopy:
Ao substituir [:] por copy.deepcopy(), garantimos que bestSolution é uma cópia totalmente independente de currentSolution. Agora, qualquer modificação em currentSolution não afeta bestSolution, garantindo que a melhor solução encontrada até aquele momento seja preservada corretamente, sem alterações inesperadas.

SyntaxError: invalid syntax (1455806134.py, line 1)